# Calculate hourly change in electricity consumption for NYC (relative to 2016-2019 average)
Hourly load data is downloaded from http://mis.nyiso.com/public/. Averages for 2016-2019 are calculated for each hour of each day and subtracted from the 2020 values to obtain a measure of the residual load. Times are already in NYC local time and takes into account daylight savings.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
from io import BytesIO
import requests
from zipfile import ZipFile
% matplotlib inline
plt.rcParams.update({'font.size': 15})


## Download Hourly Load Data
The files are contained on the server in monthly zip files. Each zip file contains day files for each day of the month.

For example:

`
http://mis.nyiso.com/public/
                |
                -- 20190601pal_csv.zip
                             |
                             -- 20190601pal.csv
                                20190602pal.csv
                                20190603pal.csv
                                       .
                                       .
                                       `

In [ ]:
# Set x-axis limits for plotting: datetime(YEAR, MONTH, DAY)
tmin = datetime(2020, 2, 7)
tmax = datetime(2020, 5, 15)

# download hourly load data (in megawatt hours)
datestr = pd.date_range('2016-01-01','2020-05-01', freq='MS').strftime("%Y%m01").tolist()
# Loop through zip files and add to list
df_all = []
for imon, monstr in enumerate(datestr):
    print(monstr)
    query_url = "http://mis.nyiso.com/public/csv/pal/"+monstr+"pal_csv.zip"
    response = requests.get(query_url)

    zip = ZipFile(BytesIO(response.content), 'r')
    # Load files into dictionary of dataframes
    dfs = {text_file.filename: pd.read_csv(zip.open(text_file.filename))
           for text_file in zip.infolist()
           if text_file.filename.endswith('.csv')}
    # Join dataframes
    dfi = pd.concat(dfs.values(), ignore_index=True)
    # Index NYC
    dfi_nyc = dfi[dfi.loc[:,'Name']=='N.Y.C.']
    # Build list of dataframes that we will concat at the end
    df_all.append(dfi_nyc)
df = pd.concat(df_all)
df.reset_index()
df.rename(columns={'Time Stamp':'Date',
                   'Load':'Load_megawatthours'},
          inplace=True)
# Convert date string to datetime
df['Date']= pd.to_datetime(df['Date'])
         

In [ ]:
# NYC stay at home 2020/3/22 8pm EST (UTC - 4)
nyc_SAH = datetime(2020,3,22,20,0) + timedelta(0,4*60*60)


## Remove yearly trend
Calculate hourly averages for years 2016-2019 and subtract from 2020 values.

In [ ]:

# Reorganize dataframe by Date (month, day, hour) with columns in years
df = df.set_index('Date')
# sort by index to make sure dates are in border
df.sort_index(axis = 0) 
# Pivot table so we can calculate hourly averages by year
pv = pd.pivot_table(df, index=[df.index.month, df.index.day, df.index.hour], columns=[df.index.year], values=['Load_megawatthours'])
# pv
pv


In [ ]:

# Calculate hourly load averages excluding 2020
load_avg = pv.loc[1:5].Load_megawatthours.iloc[:,0:4].median(axis=1)
# Save 2020 load values
load_2020 = pv.loc[1:5].Load_megawatthours.iloc[:,4]
# Calculate percent change in load used
load_2020pct_trend_removed = (load_2020-load_avg)/load_avg*100
# Remake index with datetime values
d = load_2020pct_trend_removed
d.index = pd.to_datetime('2020-'+d.index.get_level_values(0).astype(str) + '-' +
                          d.index.get_level_values(1).astype(str) + '-' +
                          d.index.get_level_values(2).astype(str),
               format='%Y-%m-%d-%H')
# Convert series to dataframe
d = d.to_frame('load_resid')
d['load_avg'] = load_avg.values
# Set date column with index values
d['Date'] = d.index
df['Date'] = df.index


## Make Plots

In [ ]:
# Plot Data
fig, ax = plt.subplots(2, 1, figsize=(15,10), sharex=True)

ax[0].plot(df.Date,df.Load_megawatthours,'-',linewidth=2,label='2020')
ax[0].plot(d.Date,d.load_avg,'-',linewidth=2,label='2016-2019 avg.')
ax[0].plot([nyc_SAH, nyc_SAH],[3000, 7000],'--r')
ax[0].set_ylabel('Total Load (Mwh)',fontsize=23)
ax[0].grid(True)
ax[0].set_xlim([tmin, tmax])
ax[0].set_ylim(3000,7000)
ax[0].legend()
ax[0].set_title('N.Y.C. Electrity Use',fontsize=23)

ax[1].plot(d.Date,d.load_resid,'-',linewidth=2)
ax[1].plot([nyc_SAH, nyc_SAH],[-30, 10],'--r')
ax[1].set_ylabel('Change in Electricity Use',fontsize=23)
ax[1].set_xlabel('Date',fontsize=23)
ax[1].grid(True)
ax[1].set_xlim([tmin, tmax])
# Rotate tick marks on x-axis
plt.setp(ax[1].get_xticklabels(), rotation=45)

plt.show()


In [ ]:
# Save hourly load averages to csv
d.to_csv('Data/load_reduction_hourly_NYC.csv', 
            columns=['Date','load_resid','load_avg'], index=False)